In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

In [11]:
from math import pow
import random
import json
from collections import defaultdict

from roaringbitmap import RoaringBitmap, MultiRoaringBitmap


class RoundRobinSamplingBitmap:
    """A bitmap sketch with fixed max size via variable sampling"""    
    
    MAX_uint32_t = 0xFFFFFFFF ## TODO find a more pythonic way to assert this
    
    # TODO to be stict to remove chance of unitialized member vars
    # TODO having a version that requires everything we need for the results of intersection, union, etc
    # TODO consider dealing with negatives in pruning
    def __init__(self, maxSamples):        
        self.maxSamples = maxSamples
        self.curSampleRate = 1   # the rate is 1/curSampleRate, so if the value is 4, it is 1/4th. start at 1/1
        self.rbm = RoaringBitmap()  

    def copy(self):
        result = RoundRobinSamplingBitmap(maxSamples=self.maxSamples)
        
        result.rbm = self.rbm.copy()
        result.curSampleRate = self.curSampleRate
        return result
        
    def _checkSize(self):          
        # first, prune out anything beyond the max because they're not in the sample
        self._prune()

        # now, as long have more than the maxSamples number, we need to grow our sampling rate and re-prune
        while len(self.rbm) > self.maxSamples:
            self.curSampleRate *= 2 # TODO consider different sampleRate scaling instead of 2
            self._prune()
    
    def _prune(self):          
        curMaxAllowed = int(self.MAX_uint32_t / self.curSampleRate)
        if self.rbm.max() > curMaxAllowed:
            self.rbm = self.rbm.clamp(0, curMaxAllowed)

    def add(self, i):          
        if i < 0: raise RuntimeError("cannot yet support negative id values")      
        self.rbm.add(i)
        self._checkSize()        
        return i
    
    def _performBitmapOp(self, other, opStr):     
                
        ## currently need to ensure the instances have the same maxSamples init values
        if self.maxSamples != other.maxSamples:
            raise RuntimeError("you cannot mix instances with different initial maxSamples values")
        ## TODO figure out how to extrapolate, if possible, to allow different init values
                     
        result = RoundRobinSamplingBitmap(maxSamples=self.maxSamples)
               
        result.rbm = getattr(self.rbm, opStr)(other.rbm)
        result.curSampleRate = max(self.curSampleRate, other.curSampleRate)
        
        result._checkSize()
        return result

    
    ## the following should work logically but havent really been tested with real data much
    def intersection(self, other):        
        return self._performBitmapOp(other, "intersection")

    def union(self, other):        
        return self._performBitmapOp(other, "union")
                 
    def difference(self, other):        
        return self._performBitmapOp(other, "difference")
    ## the above should work logically but havent really been tested with real data much

    def estimatedCardinality(self):
        return len(self.rbm)*self.curSampleRate
        
    
    ## TODO deprecate this in favor of json dumps
    def dumps(self):
        state = {
            "estCard" : int(self.estimatedCardinality()),
            "curSampleRate" : self.curSampleRate,
            "curSamples" : len(self.rbm),                       
        }        
        return json.dumps(state)
    
    def errPct(self, target):
        precision = 3
        mult = pow(10,precision+1)
        return int(mult*(self.estimatedCardinality()-target)/target)/mult
                

print(RoundRobinSamplingBitmap)

<class '__main__.RoundRobinSamplingBitmap'>


In [ ]:
import time
random.seed(time.time()*1000)


## this cell creates a rbmrr or increasing size and saves off snapshot copies along the way.
## also, for each addition it increments a count for the observed value of error between the 
## actual and estimated cardinality.

rbmrr = RoundRobinSamplingBitmap(maxSamples=2000)

iterations = 1000*1000 #100*1000*1000

snapshots = {}
NUM_SNAPSHOTS = 50
errHistogram = defaultdict(lambda:0)

for i in range(1,iterations+1):
        
    rbmrr.add(random.randint(0,rbmrr.MAX_uint32_t))
    errHistogram[rbmrr.errPct(i)] += 1
    
    if (NUM_SNAPSHOTS > 0 and i % (iterations/NUM_SNAPSHOTS) == 0): 
        print(i, rbmrr.errPct(i), rbmrr.dumps())
        snapshots[i] = rbmrr.copy()



20000 -0.0304 {"estCard": 19392, "curSampleRate": 16, "curSamples": 1212}
40000 0.0048 {"estCard": 40192, "curSampleRate": 32, "curSamples": 1256}
60000 -0.0053 {"estCard": 59680, "curSampleRate": 32, "curSamples": 1865}


In [4]:
## this cell walks the errHistogram twice to compute percentiles on observed estimation error


## the first traversal is to increment a total count and to compute a new dictory keyed 
## on the abs value of observed differences between the actual and estimated cardinality.
## it also sums the counts overall so we have the total amount.

absHist, totalCount = {}, 0
for errPct, count in errHistogram.items():
    totalCount += count
    if abs(errPct) not in absHist:
        absHist[abs(errPct)] =0

    absHist[abs(errPct)] += count
    
##  the second time walks the abs of the error to show cumulative counts.
    
cumulativeCount = 0
for absErrPct, count in sorted(absHist.items()):
    cumulativeCount += count
    print(absErrPct, count, cumulativeCount/totalCount)
    
    
## with a recent run, for example, the max error was 3.88%. that is, for all keys 
## values added sequentially from 1 and 100,000,000 using at most 10,000 samples,
## the max error was 3.88% and that occured only 488 times. the p95 of error for the
## same keys was 3.49%

    

0.0 2346 0.002346
0.0001 1225 0.003571
0.0002 1199 0.00477
0.0003 1248 0.006018
0.0004 1229 0.007247
0.0005 1206 0.008453
0.0006 1145 0.009598
0.0007 1077 0.010675
0.0008 1051 0.011726
0.0009 1100 0.012826
0.001 1082 0.013908
0.0011 1058 0.014966
0.0012 971 0.015937
0.0013 985 0.016922
0.0014 978 0.0179
0.0015 936 0.018836
0.0016 900 0.019736
0.0017 878 0.020614
0.0018 884 0.021498
0.0019 848 0.022346
0.002 851 0.023197
0.0021 861 0.024058
0.0022 834 0.024892
0.0023 836 0.025728
0.0024 882 0.02661
0.0025 835 0.027445
0.0026 880 0.028325
0.0027 883 0.029208
0.0028 880 0.030088
0.0029 906 0.030994
0.003 919 0.031913
0.0031 903 0.032816
0.0032 900 0.033716
0.0033 934 0.03465
0.0034 900 0.03555
0.0035 949 0.036499
0.0036 949 0.037448
0.0037 898 0.038346
0.0038 897 0.039243
0.0039 900 0.040143
0.004 886 0.041029
0.0041 838 0.041867
0.0042 794 0.042661
0.0043 770 0.043431
0.0044 744 0.044175
0.0045 685 0.04486
0.0046 657 0.045517
0.0047 651 0.046168
0.0048 654 0.046822
0.0049 648 0.04747
0.0

0.0596 584 0.614627
0.0597 658 0.615285
0.0598 601 0.615886
0.0599 455 0.616341
0.06 474 0.616815
0.0601 500 0.617315
0.0602 471 0.617786
0.0603 687 0.618473
0.0604 664 0.619137
0.0605 653 0.61979
0.0606 789 0.620579
0.0607 835 0.621414
0.0608 824 0.622238
0.0609 843 0.623081
0.061 863 0.623944
0.0611 818 0.624762
0.0612 791 0.625553
0.0613 946 0.626499
0.0614 1068 0.627567
0.0615 852 0.628419
0.0616 1050 0.629469
0.0617 1052 0.630521
0.0618 990 0.631511
0.0619 1066 0.632577
0.062 1092 0.633669
0.0621 1072 0.634741
0.0622 1232 0.635973
0.0623 1371 0.637344
0.0624 1504 0.638848
0.0625 1598 0.640446
0.0626 1478 0.641924
0.0627 1634 0.643558
0.0628 1781 0.645339
0.0629 1997 0.647336
0.063 2290 0.649626
0.0631 2310 0.651936
0.0632 2277 0.654213
0.0633 2367 0.65658
0.0634 2565 0.659145
0.0635 2979 0.662124
0.0636 2995 0.665119
0.0637 2746 0.667865
0.0638 2824 0.670689
0.0639 2881 0.67357
0.064 2817 0.676387
0.0641 2797 0.679184
0.0642 2725 0.681909
0.0643 3008 0.684917
0.0644 3127 0.688044


In [5]:

## here we use the snapshots to compare error in intersect estimates

largestSnapshotSize = max(snapshots.keys())
largestSnapshot = snapshots[largestSnapshotSize]

print("intersecting with the largest snapshot with %d entries\n" % largestSnapshotSize)

intersectErrPcts = []
for k,v in snapshots.items():
    intersect = largestSnapshot.intersection(v)
    intersectErrPcts.append(intersect.errPct(k))
    print(k, intersect.errPct(k), intersect.dumps())





intersecting with the largest snapshot with 1000000 entries

20000 -0.1808 {"estCard": 16384, "curSampleRate": 1024, "curMax": 3743395, "getCurMaxAllowed": 4194303}
40000 -0.0528 {"estCard": 37888, "curSampleRate": 1024, "curMax": 4113348, "getCurMaxAllowed": 4194303}
60000 -0.0784 {"estCard": 55296, "curSampleRate": 1024, "curMax": 4113348, "getCurMaxAllowed": 4194303}
80000 -0.1936 {"estCard": 64512, "curSampleRate": 1024, "curMax": 4113348, "getCurMaxAllowed": 4194303}
100000 -0.2115 {"estCard": 78848, "curSampleRate": 1024, "curMax": 4166001, "getCurMaxAllowed": 4194303}
120000 -0.249 {"estCard": 90112, "curSampleRate": 1024, "curMax": 4166001, "getCurMaxAllowed": 4194303}
140000 -0.21 {"estCard": 110592, "curSampleRate": 1024, "curMax": 4166633, "getCurMaxAllowed": 4194303}
160000 -0.136 {"estCard": 138240, "curSampleRate": 1024, "curMax": 4167585, "getCurMaxAllowed": 4194303}
180000 -0.1239 {"estCard": 157696, "curSampleRate": 1024, "curMax": 4167585, "getCurMaxAllowed": 4194303}

In [6]:
mrb = MultiRoaringBitmap([v.rbm.freeze() for v in snapshots.values()])
print("avg snapshot size: %d bytes" % (mrb.bufsize()/len(snapshots.values())))

snapshotBufsize = [(k,v.curSampleRate,MultiRoaringBitmap([v.rbm]).bufsize()) for (k,v) in snapshots.items()]
sorted(snapshotBufsize)

avg snapshot size: 7832 bytes


[(20000, 32, 26560),
 (40000, 64, 23968),
 (60000, 64, 31040),
 (80000, 128, 17600),
 (100000, 128, 19648),
 (120000, 128, 20960),
 (140000, 256, 11040),
 (160000, 256, 11648),
 (180000, 256, 11904),
 (200000, 256, 12064),
 (220000, 256, 12352),
 (240000, 256, 12544),
 (260000, 256, 12544),
 (280000, 512, 6368),
 (300000, 512, 6400),
 (320000, 512, 6400),
 (340000, 512, 6464),
 (360000, 512, 6464),
 (380000, 512, 6464),
 (400000, 512, 6464),
 (420000, 512, 6464),
 (440000, 512, 6496),
 (460000, 512, 6496),
 (480000, 512, 6496),
 (500000, 512, 6560),
 (520000, 512, 6560),
 (540000, 1024, 3328),
 (560000, 1024, 3328),
 (580000, 1024, 3328),
 (600000, 1024, 3328),
 (620000, 1024, 3328),
 (640000, 1024, 3328),
 (660000, 1024, 3328),
 (680000, 1024, 3360),
 (700000, 1024, 3360),
 (720000, 1024, 3360),
 (740000, 1024, 3392),
 (760000, 1024, 3456),
 (780000, 1024, 3488),
 (800000, 1024, 3520),
 (820000, 1024, 3552),
 (840000, 1024, 3584),
 (860000, 1024, 3648),
 (880000, 1024, 3680),
 (900000